# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
csvpath = '../output_data/cities.csv'
cities_df = pd.read_csv(csvpath, index_col=0)
cities_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
City_ID,,,,,,,,,
0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...
543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
locations = cities_df[["Lat", "Lng"]]
humidity = cities_df['Humidity'].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
cities_df.describe()

,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
count,548.000000,5.480000e+02,548.000000,548.000000,548.000000,548.000000,548.000000
mean,49.405109,1.558379e+09,67.671533,19.665146,20.675529,64.970785,8.672682
std,36.914201,7.820486e+01,23.697626,33.623560,90.307948,18.704318,5.865974
min,0.000000,1.558378e+09,4.000000,-54.810000,-173.230000,4.920000,0.540000
25%,13.750000,1.558379e+09,52.000000,-6.267500,-61.015000,53.970000,4.680000
50%,46.500000,1.558379e+09,71.000000,21.390000,21.240000,64.990000,7.010000
75%,81.000000,1.558379e+09,86.000000,47.887500,102.937500,80.600000,11.410000
max,100.000000,1.558379e+09,290.000000,78.220000,179.320000,104.100000,46.080000


In [8]:
cities_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Cloudiness,548.0,4.940511e+01,36.914201,0.000000e+00,1.375000e+01,4.650000e+01,8.100000e+01,1.000000e+02
Date,548.0,1.558379e+09,78.204862,1.558378e+09,1.558379e+09,1.558379e+09,1.558379e+09,1.558379e+09
Humidity,548.0,6.767153e+01,23.697626,4.000000e+00,5.200000e+01,7.100000e+01,8.600000e+01,2.900000e+02
Lat,548.0,1.966515e+01,33.623560,-5.481000e+01,-6.267500e+00,2.139000e+01,4.788750e+01,7.822000e+01
Lng,548.0,2.067553e+01,90.307948,-1.732300e+02,-6.101500e+01,2.124000e+01,1.029375e+02,1.793200e+02
Max Temp,548.0,6.497078e+01,18.704318,4.920000e+00,5.397000e+01,6.499000e+01,8.060000e+01,1.041000e+02
Wind Speed,548.0,8.672682e+00,5.865974,5.400000e-01,4.680000e+00,7.010000e+00,1.141000e+01,4.608000e+01


In [10]:
ideal_cities_df = cities_df.loc[(cities_df['Max Temp'] > 65)  &
                                (cities_df['Max Temp'] < 75) & 
                                (cities_df['Wind Speed'] < 40) &
                                (cities_df['Cloudiness'] == 32)
                       ]

hotel_df = ideal_cities_df.dropna()

print(f'There are {len(hotel_df)} ideal cities for vacation')
hotel_df

There are 1 ideal cities for vacation


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
City_ID,,,,,,,,,
296,khash,32,IR,1558378844,43,28.22,61.21,68.28,12.5


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
Latitude = hotel_df['Lat'].tolist()
Longitude = hotel_df['Lng'].tolist()
hotel_lst = []

for i in range(len(hotel_df)):
    target_coordinates = f"{Latitude[i]}, {Longitude[i]}"
    target_search = 'hotel'
    target_type = "hotel"
    target_radius = 5000

    params = {
        "location": target_coordinates,
        "types": target_type,
        "keyword": target_search,
        "radius": target_radius,
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    
    try:
        
        response = requests.get(base_url, params=params).json()
        hotel_lst.append(response['results'][0]['name'])
    
    except:

        hotel_lst.append('NO HOTEL FOUND')

hotel_df['Hotel Name'] = hotel_lst
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
City_ID,,,,,,,,,,
296,khash,32,IR,1558378844,43,28.22,61.21,68.28,12.5,خوابگاه دانشجویی خاش


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [17]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))